# Tideway Query Template (BMC Discovery)

Use this notebook to run a TWQL query against one or more BMC Discovery appliances and load the results into pandas.


## Requirements

This template relies on `tideway`, `pandas`, and `PyYAML`. Uncomment the install cell below if needed.


In [ ]:
# %pip install -q tideway pandas pyyaml

import pandas as pd
import yaml
from pathlib import Path
from typing import Any, Dict, Optional
import xml.etree.ElementTree as ET
import tideway


In [ ]:
QUERY_TITLE = 'Discovery Run Analysis'
APPLIANCE_NAMES = ['prod', 'dev']  # Edit as needed; empty list tries default entry


In [ ]:
## Configuration

Name the query to load from `dismal_queries.xml`, configure the appliance list, and adjust any column mappings as needed.


In [ ]:
COLUMN_MAP: Dict[str, str] = {
    # Example: 'Original Column Name': 'Renamed Column'
}

NUMERIC_COLUMNS = []  # Columns to coerce to integer (if present)


In [ ]:
def load_query(title: str) -> str:
    xml_path = Path('DisMAL/queries/dismal_queries.xml')
    tree = ET.parse(xml_path)
    root = tree.getroot()
    for query in root.findall('query'):
        if query.get('title') == title:
            search = query.find('search')
            if search is not None and search.text:
                return search.text
    raise ValueError(f"Query '{title}' not found in dismal_queries.xml")

QUERY_TEXT = load_query(QUERY_TITLE)


## Appliance Selection

This section initialises the appliances you want to query. Update `APPLIANCE_NAMES` above or leave it empty to fall back to the default entry from `config.yaml`.


In [ ]:
def load_config_params(start: Path, appliance_name: Optional[str] = None, appliance_index: int = 0) -> Dict[str, Any]:
    def _find_repo_root(start_path: Path) -> Path:
        for p in [start_path] + list(start_path.parents):
            if (p / 'config.yaml').exists():
                return p
        return start_path.parent

    repo_root = _find_repo_root(start)
    config_path = repo_root / 'config.yaml'
    with open(config_path, 'r') as fh:
        cfg = yaml.safe_load(fh) or {}

    apps = cfg.get('appliances') or []
    selected = None
    if isinstance(apps, list) and apps:
        if appliance_name:
            selected = next((a for a in apps if a.get('name') == appliance_name), None)
            if selected is None:
                raise ValueError(f"No appliance named '{appliance_name}' in config.yaml")
        else:
            try:
                selected = apps[int(appliance_index)]
            except Exception:
                selected = apps[0]

    target = ((selected or {}).get('target') or cfg.get('target') or '').strip()
    if not target:
        raise ValueError('config.yaml missing "target"')

    token = (((selected or {}).get('token') or cfg.get('token') or '').strip())
    token_file = (selected or {}).get('token_file') or cfg.get('token_file') or cfg.get('f_token')
    if not token and token_file:
        tf_path = Path(token_file)
        if not tf_path.is_absolute():
            tf_path = repo_root / tf_path
        with open(tf_path, 'r') as tf:
            token = tf.read().strip()
    if not token:
        raise ValueError('API token not found in config.yaml (token or token_file)')

    api_version = str((selected or {}).get('api_version') or cfg.get('api_version') or 'v1.14')
    verify_ssl = bool((selected or {}).get('verify_ssl', cfg.get('verify_ssl', True)))

    sanitized = target.replace('.', '_').replace(':', '_').replace('/', '_')
    output_dir = repo_root / f'output_{sanitized}'
    output_dir.mkdir(parents=True, exist_ok=True)

    return {
        'repo_root': repo_root,
        'config_path': config_path,
        'cfg': cfg,
        'selected': selected,
        'target': target,
        'token': token,
        'api_version': api_version,
        'verify_ssl': verify_ssl,
        'output_dir': output_dir,
    }


In [ ]:
def init_appliance(appliance_name: Optional[str] = "prod") -> Dict[str, Any]:
    params = load_config_params(Path.cwd(), appliance_name=appliance_name)
    target = params["target"]
    api_number = params["api_version"].lstrip('v')

    print('Base Host     :', target)
    print('API Version   :', api_number)
    print('Verify SSL    :', params["verify_ssl"])
    print('Output folder :', params["output_dir"])

    app = tideway.appliance(target, params["token"], api_version=api_number, ssl_verify=params["verify_ssl"])
    twsearch = app.data()

    try:
        about = app.api_about
        print('Appliance reachable:', getattr(about, 'status_code', 'ok'))
    except Exception as exc:
        print('Warning: failed to contact appliance /api/about:', exc)

    return {
        "params": params,
        "target": target,
        "app": app,
        "search": twsearch,
        "api_version": api_number,
        "output_dir": params["output_dir"],
        "name": (params["selected"] or {}).get("name") or (appliance_name or target),
    }


In [ ]:
instances: list[Dict[str, Any]] = []

def _attempt_init(name: Optional[str]):
    label = name if name else 'default'
    try:
        print(f"Initialise {label.capitalize()}:")
        inst = init_appliance(name)
        instances.append(inst)
    except Exception as exc:
        print(f"Skipping {label}: {exc}")

if APPLIANCE_NAMES:
    for name in APPLIANCE_NAMES:
        _attempt_init(name)
else:
    _attempt_init(None)

if not instances:
    raise RuntimeError('No appliances could be initialised from config.yaml')


In [ ]:
def fetch_query_results(instance: Dict[str, Any]) -> pd.DataFrame:
    search = instance['search']
    try:
        results = search.search({'query': QUERY_TEXT}, format='object', limit=200)
    except Exception as exc:
        print(f"Query failed for {instance['target']}: {exc}")
        results = []

    df = pd.DataFrame(results) if results else pd.DataFrame()
    df.insert(0, 'Discovery Instance', instance['target'])

    if COLUMN_MAP:
        df = df.rename(columns=COLUMN_MAP)

    return df


def convert_numeric_columns(frame: pd.DataFrame) -> pd.DataFrame:
    converted = frame.copy()
    for col in NUMERIC_COLUMNS:
        if col in converted.columns:
            converted[col] = pd.to_numeric(converted[col], errors='coerce').astype('Int64')
    return converted


## Fetch query results

Run the configured TWQL query against each appliance and load the data into a pandas DataFrame.


In [ ]:
results_by_instance: list[dict[str, Any]] = []
for inst in instances:
    df = fetch_query_results(inst)
    results_by_instance.append({'instance': inst, 'data': df})
    print(inst['target'])
    if not df.empty:
        display(df.head(10))
    else:
        print('No rows returned.')


## Inspect sample rows

Show a few relevant columns such as labels, timing and counts when present.


In [ ]:
processed_results: list[dict[str, Any]] = []
for item in results_by_instance:
    inst = item['instance']
    df = convert_numeric_columns(item['data'])
    other_cols = [c for c in df.columns if c != 'Discovery Instance']
    if other_cols:
        df = df[['Discovery Instance'] + other_cols]
    processed_results.append({'instance': inst, 'data': df})

if processed_results:
    combined_results = pd.concat([item['data'] for item in processed_results], ignore_index=True)
else:
    combined_results = pd.DataFrame(columns=['Discovery Instance'])

display(combined_results.head(10))


## Save to CSV

Persist the combined dataset to the project output directory (`output_<target>`).

This cell inserts a leading `Discovery Instance` column and casts any configured numeric columns before writing each appliance's CSV.


In [ ]:
for item in processed_results:
    inst = item['instance']
    df = item['data']
    outfile = QUERY_TITLE.lower().replace(' ', '_') + '.csv'
    output_csv = inst['output_dir'] / outfile
    df.to_csv(output_csv, index=False)
    print(f'Saved to {output_csv}')


---
### Notes
- If your appliance uses a self-signed certificate, set `verify_ssl` to `False` in `config.yaml`.
- Adjust `QUERY_TITLE`, `COLUMN_MAP`, and `NUMERIC_COLUMNS` at the top as needed.
- Use the resulting DataFrame for further analysis or exporting.
